ETL - Air Quality Index - aqicn.org

This notebook contains the code to extract Air Quality Index data from desired geolocation.
First, we will connect to aqicn API and extract the data using REST.
Second, we are going to convert the input dictionary into a dataframe.
Third, we are going to load the processed data in a sagemaker feature store or DynamoDB

# Extract

In [23]:
import pandas as pd
import requests
from datetime import datetime
import pytz
import boto3
from decimal import Decimal

In [3]:
response= requests.get(
    url="https://api.waqi.info/feed/shanghai",
    params={
        "token": "4160d19c73429aaee7d89467b128c082b3c6b868"
    }
).json()

In [5]:
response

{'status': 'ok',
 'data': {'aqi': 46,
  'idx': 1437,
  'attributions': [{'url': 'http://106.37.208.233:20035/emcpublish/',
    'name': 'China National Urban air quality real-time publishing platform (全国城市空气质量实时发布平台)'},
   {'url': 'https://china.usembassy-china.org.cn/embassy-consulates/shanghai/air-quality-monitor-stateair/',
    'name': 'U.S. Consulate Shanghai Air Quality Monitor'},
   {'url': 'https://sthj.sh.gov.cn/',
    'name': 'Shanghai Environment Monitoring Center(上海市环境监测中心)'},
   {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
  'city': {'geo': [31.2047372, 121.4489017],
   'name': 'Shanghai (上海)',
   'url': 'https://aqicn.org/city/shanghai',
   'location': ''},
  'dominentpol': 'pm25',
  'iaqi': {'co': {'v': 3.7},
   'h': {'v': 72},
   'no2': {'v': 13.3},
   'o3': {'v': 30.9},
   'p': {'v': 1020},
   'pm10': {'v': 18},
   'pm25': {'v': 46},
   'so2': {'v': 3.6},
   't': {'v': 15},
   'w': {'v': 2.5}},
  'time': {'s': '2024-10-29 21:00:00',
   'tz':

In [70]:
response= requests.get(
    url="https://api.waqi.info/feed/geo:19.42673333;-99.12543333/",
    params={
        "token": "4160d19c73429aaee7d89467b128c082b3c6b868"
    }
).json()

In [71]:
response

{'status': 'ok',
 'data': {'aqi': 85,
  'idx': 404,
  'attributions': [{'url': 'http://www.aire.df.gob.mx/',
    'name': 'SINAICA - Sistema Nacional de Información de la Calidad del Aire en Mexico',
    'logo': 'Mexico-CuidaMexico.png'},
   {'url': 'http://sinaica.inecc.gob.mx/',
    'name': 'INECC - Instituto Nacional de Ecolog&iacute;a y Cambio Clim&aacute;tico',
    'logo': 'Mexico-INECC.png'},
   {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
  'city': {'geo': [19.42461, -99.119594],
   'name': 'Merced, México, Mexico',
   'url': 'https://aqicn.org/city/mexico/mexico/merced',
   'location': ''},
  'dominentpol': 'pm25',
  'iaqi': {'co': {'v': 8.9},
   'dew': {'v': 10},
   'h': {'v': 87},
   'no2': {'v': 24.1},
   'o3': {'v': 0.8},
   'p': {'v': 1027},
   'pm10': {'v': 42},
   'pm25': {'v': 85},
   'so2': {'v': 8.6},
   't': {'v': 12},
   'w': {'v': 3}},
  'time': {'s': '2024-10-29 06:00:00',
   'tz': '-06:00',
   'v': 1730181600,
   'iso': '2024-10-29T06

# Transform

Save only the Air Quality Index for each pollutant

In [72]:
saved_cols = ["co","no2","pm10","pm25","o3","so2"]

In [73]:
pollutants = response["data"]["iaqi"]
pollutants

{'co': {'v': 8.9},
 'dew': {'v': 10},
 'h': {'v': 87},
 'no2': {'v': 24.1},
 'o3': {'v': 0.8},
 'p': {'v': 1027},
 'pm10': {'v': 42},
 'pm25': {'v': 85},
 'so2': {'v': 8.6},
 't': {'v': 12},
 'w': {'v': 3}}

In [74]:

for k in pollutants.keys():
    pollutants[k] = round(Decimal(pollutants[k]["v"]),3)

In [75]:
data_json = {name : pollutants[name] for name in saved_cols}
data_json["datetime"] = datetime.now(pytz.timezone("America/Mexico_City")).strftime("%d/%m/%Y, %H:%M:%S")

air_data_location = "Mexico City"
data_json["location"] = air_data_location

In [84]:
# get boto3 dynamoDB client for desired table
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('AirQualityIndexRecords')



data_json["id"] = table.scan()["Count"]


In [85]:
data_json

{'co': Decimal('8.900'),
 'no2': Decimal('24.100'),
 'pm10': Decimal('42.000'),
 'pm25': Decimal('85.000'),
 'o3': Decimal('0.800'),
 'so2': Decimal('8.600'),
 'datetime': '29/10/2024, 08:05:33',
 'location': 'Mexico City',
 'id': 0}

# Load

In [86]:
table.put_item(
    Item=data_json
)

{'ResponseMetadata': {'RequestId': 'LJ2Q8RVCSVJM8RJVKULD46B5RVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 29 Oct 2024 14:07:08 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'LJ2Q8RVCSVJM8RJVKULD46B5RVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [94]:
table.get_item(
    Key={"id" : 0, "location" : "Mexico City"}
)

{'Item': {'no2': Decimal('24.1'),
  'datetime': '29/10/2024, 08:05:33',
  'location': 'Mexico City',
  'pm25': Decimal('85'),
  'o3': Decimal('0.8'),
  'so2': Decimal('8.6'),
  'id': Decimal('0'),
  'co': Decimal('8.9'),
  'pm10': Decimal('42')},
 'ResponseMetadata': {'RequestId': 'PM9RGTDV1H6GBDTOG85PFL8GMBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 29 Oct 2024 14:11:03 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '203',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'PM9RGTDV1H6GBDTOG85PFL8GMBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2591864421'},
  'RetryAttempts': 0}}